In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2026-02-16 08:11:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2026-02-16 08:11:02 (15.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt','r',encoding='utf-8') as f:
  text = f.read();

In [3]:
print(f'Lenght of the dataset is:{len(text)}')

Lenght of the dataset is:1115394


In [4]:
chars = sorted(list(set(text)))
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [5]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hi there"))
print(decode(encode("hi there")))


[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [6]:
import torch
data = torch.tensor(encode(text),dtype = torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size+1]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when context is {context} the taget is {target}")

when context is tensor([18]) the taget is 47
when context is tensor([18, 47]) the taget is 56
when context is tensor([18, 47, 56]) the taget is 57
when context is tensor([18, 47, 56, 57]) the taget is 58
when context is tensor([18, 47, 56, 57, 58]) the taget is 1
when context is tensor([18, 47, 56, 57, 58,  1]) the taget is 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]) the taget is 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the taget is 58


In [36]:
torch.manual_seed(1337)
#batch_size will determine the number of sequences to process parallelly
batch_size = 4
#block_size will determine the context lenght of the prediction
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  """
  We are picking random index to make blocks for each sequence of batch_size
  Then stack the extracted sequence using torch.stack
  """
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x.to(device),y.to(device)

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]], device='cuda:0')


In [11]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when context is {context.tolist()} the target is {target}")

when context is [24] the target is 43
when context is [24, 43] the target is 58
when context is [24, 43, 58] the target is 5
when context is [24, 43, 58, 5] the target is 57
when context is [24, 43, 58, 5, 57] the target is 1
when context is [24, 43, 58, 5, 57, 1] the target is 46
when context is [24, 43, 58, 5, 57, 1, 46] the target is 43
when context is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when context is [44] the target is 53
when context is [44, 53] the target is 56
when context is [44, 53, 56] the target is 1
when context is [44, 53, 56, 1] the target is 58
when context is [44, 53, 56, 1, 58] the target is 46
when context is [44, 53, 56, 1, 58, 46] the target is 39
when context is [44, 53, 56, 1, 58, 46, 39] the target is 58
when context is [44, 53, 56, 1, 58, 46, 39, 58] the target is 1
when context is [52] the target is 58
when context is [52, 58] the target is 1
when context is [52, 58, 1] the target is 58
when context is [52, 58, 1, 58] the target is 46
when context

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self, idx, targets=None):
    # idx and targets are both (B,T) tensor and C is the vocab_size
    logits = self.token_embedding_table(idx) #(B,T,C) Batch by Time by Channel

    if targets is None:
      loss = None
    else:
    # The funtional cross_entrophy loss takes (B,C)
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      target = targets.view(B*T)
      loss = F.cross_entropy(logits,target)

    return logits,loss

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self(idx)
      logits = logits[:,-1,:] # becomes (B, C)
      probs = F.softmax(logits,dim=-1) # (B, C)
      idx_next = torch.multinomial(probs, num_samples=1) #(B, 1)
      idx = torch.cat((idx,idx_next),dim=1) #(B, T+1)
    return idx

vocab_size = 65
m = BigramModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1,1),dtype = torch.long)
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [13]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(10000):
  xb,yb = get_batch('train')

  logits,loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.382369041442871


In [15]:
print(decode(m.generate(idx,max_new_tokens=500)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


###Self-attention

In [16]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

getting hold of attention

In [17]:
"""
1) Building intuition:
    According to Andrej Karpathy- "If we want our current token to communicate to
    the past, the simplest way is to just do an average of all the previous tokens"

"""
# We want x[b,t] = mean{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
      xprev = x[b,:t+1] #(t,C)
      xbow[b,t] = torch.mean(xprev,0)

In [18]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [19]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

Getting fast using Matrix manipulation

In [20]:
torch.manual_seed(42)
a = torch.ones(3,3)
print('a: ')
print(a)

a_tril = torch.tril(a)
print('a_tril: ')
print(a_tril)

a_prime = a_tril/torch.sum(a_tril,1,keepdim=True)
print('a_prime: ')
print(a_prime)

b = torch.randint(0,10,(3,2)).float()
c = a_prime @ b

print("-------------")
print('b: ')
print(b)
print("-------------")
print('c: ')
print(c)
print("Here we got the average of the previous tokens of vector b")

a: 
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
a_tril: 
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
a_prime: 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
-------------
b: 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
-------------
c: 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
Here we got the average of the previous tokens of vector b


In [21]:
#version 2
w = torch.tril(torch.ones(T, T))
w = w / w.sum(1, keepdim=True)
xbow2 = w @ x # (B, T, T) @ (B, T, C) -> (B, T, C)

In [22]:
#version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
w = torch.zeros((T,T))
w = w.masked_fill(tril == 0, float('-inf'))
w = F.softmax(w, dim=1)
xbow3 = w @ x
torch.allclose(xbow2, xbow3)

True

In [23]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [24]:
w = torch.zeros((T,T))
w = w.masked_fill(tril == 0, float('-inf'))
w = F.softmax(w, dim=-1)
w

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [25]:
#version 4: using self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

# single head self-attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)
k = key(x) #(B,T,16)
q = query(x) #(B,T,16)
v = value(x) #(B,T,16)
w = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) ----> (B,T,T)

tril = torch.tril(torch.ones(T,T))
w = w.masked_fill(tril == 0, float('-inf')) # this is to prevent the future token to communicate
w = F.softmax(w, dim =-1)
out = w @ v #(B,T,16)
out.shape

torch.Size([4, 8, 16])

In [26]:
w[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

###Understanding concept of Scaled dot product Attention


In [27]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
w = q @ k.transpose(-2,-1)
print(f"var of q: {q.var()}")
print(f"var of k: {k.var()}")
print("--------------------------------")
print("variance when not scaled: ")
print(f"w: {w.var()}")
print("--------------------------------")
print("variance when scaled: ")
w = q @ k.transpose(-2,-1) * head_size**-0.5
print(f"scaled w: {w.var()}")

var of q: 1.0700464248657227
var of k: 1.044861912727356
--------------------------------
variance when not scaled: 
w: 17.46897315979004
--------------------------------
variance when scaled: 
scaled w: 1.0918108224868774


###Building Attention class

In [38]:
torch.manual_seed(1337)
batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layers = 6
dropout = 0.2
print(device)

cuda


In [39]:
class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    self.w_key = nn.Linear(n_embd,head_size,bias=False)
    self.w_query = nn.Linear(n_embd, head_size,bias = False)
    self.w_value = nn.Linear(n_embd,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    B,T,C = x.shape
    k = self.w_key(x)
    q = self.w_query(x)
    v = self.w_value(x)
    w = q @ k.transpose(-2,-1) * C**-0.5
    w = w.masked_fill(self.tril[:T,:T]==0,float('-inf'))
    w = F.softmax(w,dim=-1)
    w = self.dropout(w)
    out = w @ v
    return out

In [40]:
class MultiheadAttention(nn.Module):
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.projection = nn.Linear(n_embd,n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads],dim=-1)
    out = self.dropout(self.projection(out))
    return out


In [41]:
class FeedForward(nn.Module):
  def __init__(self,n_embd):
    super().__init__()
    self.ff = nn.Sequential(
        nn.Linear(n_embd,4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd,n_embd),
        nn.Dropout(dropout),
    )

  def forward(self,x):
    return self.ff(x)

In [42]:
class Block(nn.Module):
  def __init__(self,n_embd,n_head):
    super().__init__()
    head_size = n_embd//n_head
    self.sa = MultiheadAttention(n_head,head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self,x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [43]:
class LayerNorm:
  def __init__(self,dim,eps=1e-5):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self,x):
    xmean = x.mean(1,keepdim=True)
    xvar = x.var(1,keepdim=True)
    xhat = (x-xmean)/torch.sqrt(xvar+self.eps)
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma,self.beta]


In [44]:

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split)
      logits,loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

class DTransformer(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layers)])
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B,T = idx.shape
    # idx and targets are both (B,T) tensor and C is the vocab_size
    tok_emb = self.token_embedding_table(idx)
    pos_emd = self.position_embedding_table(torch.arange(T,device=device))
    x = tok_emb + pos_emd
    x = self.blocks(x)
    logits = self.lm_head(x)

    if targets is None:
      loss = None
    else:
    # The funtional cross_entrophy loss takes (B,C)
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      target = targets.view(B*T)
      loss = F.cross_entropy(logits,target)

    return logits,loss

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits,loss = self(idx_cond)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx,idx_next),dim=1)
    return idx

model = DTransformer(vocab_size)
m = model.to(device)

optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train loss: {losses['train']}, val loss: {losses['val']}")

  xb,yb = get_batch('train')

  logits, loss = model(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step: 0, train loss: 4.475303649902344, val loss: 4.47085428237915
step: 500, train loss: 2.085021734237671, val loss: 2.149052381515503
step: 1000, train loss: 1.662850022315979, val loss: 1.8235116004943848
step: 1500, train loss: 1.4917786121368408, val loss: 1.6833006143569946
step: 2000, train loss: 1.385757327079773, val loss: 1.6078898906707764
step: 2500, train loss: 1.3164312839508057, val loss: 1.5616989135742188
step: 3000, train loss: 1.2613117694854736, val loss: 1.5310593843460083
step: 3500, train loss: 1.2162723541259766, val loss: 1.5082234144210815
step: 4000, train loss: 1.1774441003799438, val loss: 1.4936163425445557
step: 4500, train loss: 1.13816237449646, val loss: 1.4830631017684937


In [45]:
context = torch.zeros((1,1), dtype=torch.long, device = device)
print(decode(m.generate(context,max_new_tokens=500)[0].tolist()))


But with prisophecal to seek think you,
mercy; lamentables, the king ha fifty of the groan;
Where good first and lewan purstiled you no more:
But for my soldier when shall have wear them?

First Citizen:
My lord, how chair lack upon, discharity
Kill'd my hence willow it. Canollo, come! her, cousin,
And till I claim with his plic in to truth,
For came to make unnurstance that loog-tren's
Of you weighs, I will po'd with my strong.
You know not what, I do say them you!

LUCIO:
O three you strike no


In [46]:
open('output.txt','w').write(decode(m.generate(context,max_new_tokens=10000)[0].tolist()))

10001

In [47]:
model_path = 'model.pth'
torch.save(m.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to model.pth
